In [ ]:
%load_ext autoreload
%autoreload 2


import geodatasets
import geopandas as gpd
import numpy as np
from mapclassify import classify

import lonboard.geopandas  # noqa: F401
from lonboard import PolygonLayer

In [ ]:
%load_ext watermark
%watermark -a 'eli knaap'  -iv

In [ ]:
gdf = gpd.read_file(geodatasets.get_path("geoda.milwaukee1"))

In [ ]:
gdf = gdf.to_crs(gdf.estimate_utm_crs())

In [ ]:
gdf = gdf[["HH_INC", "geometry"]]

## Simple Map

In [ ]:
gdf.explore()

## boorish (classless :P) choropleth

In [ ]:
gdf.explore("HH_INC", tiles="CartoDB Darkmatter")

In [ ]:
gdf.lb.explore(
    column="HH_INC",
)

In [ ]:
gdf.assign(catinc=gdf.HH_INC.astype("str")).lb.explore(
    column="catinc",
    categorical=True,
)

## simple classified choropleth

In [ ]:
gdf.explore(
    column="HH_INC",
    scheme="quantiles",
    k=6,
    cmap="YlOrBr",
)

In [ ]:
gdf.lb.explore(
    column="HH_INC",
    cmap="YlOrBr",
    scheme="quantiles",
    alpha=0.5,
    layer_kwargs={"opacity": 1},
)

In [ ]:
gdf.lb.explore(
    column="HH_INC",
    cmap="YlOrBr",
    scheme="quantiles",
    alpha=1,
    layer_kwargs={"opacity": 0.5},
)

### custom classification

In [ ]:
classify_kwds = {"bins": [20000, 40000, 800000, 2000000]}

gdf.explore(
    column="HH_INC",
    cmap="YlOrBr",
    scheme="user_defined",
    classification_kwds=classify_kwds,
)

In [ ]:
gdf.lb.explore(
    column="HH_INC",
    cmap="YlOrBr",
    scheme="user_defined",
    classification_kwds=classify_kwds,
)

## compose with lines and points

In [ ]:
linedf = gpd.GeoDataFrame(geometry=gdf.boundary)
linedf["len"] = linedf.length

In [ ]:
m = linedf.explore(
    column="len",
    cmap="viridis_r",
    scheme="quantiles",
    k=10,
    tiles="CartoDB Darkmatter",
)
gdf.set_geometry(gdf.centroid).explore(color="magenta", m=m)
m

In [ ]:
m = linedf.lb.explore(column="len", cmap="viridis_r", scheme="quantiles", k=10)
gdf.set_geometry(gdf.centroid).lb.explore(color="magenta", m=m)

# cant do this part unless you use fit_bounds?
m.set_view_state(zoom=10.5, longitude=-87.85, latitude=43.05)
m

## categorical

In [ ]:
# random categories
gdf["i"] = gdf.index.values

In [ ]:
gdf.explore(
    "i",
    categorical=True,
)

In [ ]:
gdf.assign(i=gdf["i"].astype(str)).lb.explore(
    column="i",
    # categorical=True,
)

In [ ]:
gdf.assign(icat=gdf["i"].astype("category")).lb.explore(
    column="icat",
    categorical=True,
)

In [ ]:
categories = classify(gdf.HH_INC, scheme="quantiles", k=5).yb

In [ ]:
# categories that are actually quantiles
gdf["q5"] = categories
gdf["q5"] = gdf["q5"].astype("category")

In [ ]:
gdf.explore("q5", categorical=True, cmap="tab20b", tiles="CartoDB Positron")

In [ ]:
m = gdf.lb.explore(
    column="q5",
    categorical=True,
    cmap="tab20b",
    nan_color=[0, 0, 0, 0],
    tiles="CartoDB Positron",
)
m

In [ ]:
m = gdf.lb.explore(column="q5", categorical=True, cmap="RdBu", alpha=0.5)

In [ ]:
m

## animated choropleth

In [ ]:
m = gdf.lb.explore(
    categorical=True,
    color="orange",
    map_kwargs={"show_side_panel": True},
)
m

In [ ]:
m.layers[0].get_fill_color = "blue"

without rerendering the map you can update the color

(this updates *both* maps, including the cell above)

In [ ]:
m

In [ ]:
from time import sleep

for _ in range(5):
    for color in ["yellow", "red", "blue"]:
        m.layers[0].get_fill_color = color
        sleep(0.3)
    sleep(1)

change color in a timed loop like a gif

In [ ]:
from ipywidgets import widget
from mapclassify import classify
from mapclassify.util import get_color_array
from matplotlib import colormaps

In [ ]:
def choro(
    vals: np.ndarray,
    classifier: str,
    cmap: str,
    k: int,
    layer: PolygonLayer,
) -> widget:
    layer.get_fill_color = get_color_array(vals, classifier, k=k, cmap=cmap)

In [ ]:
m = gdf.lb.explore()
m

In [ ]:
from mapclassify._classify_API import _classifiers

In [ ]:
from ipywidgets import fixed, interact

interact(
    choro,
    vals=fixed(gdf.HH_INC),
    classifier=list(_classifiers.keys()),
    k=range(3, 10),
    cmap=list(colormaps.keys()),
    layer=fixed(m.layers[0]),
)

In [ ]:
gdf.loc[0:200, "HH_INC"] = np.nan

m = gdf.lb.explore(
    column="HH_INC",
    cmap="Accent",
    nan_color=[0, 0, 0, 0],
)
m

## Additional features not yet implemented

The cells below may be considered via additional keyword arguments to the `explore` method. If these options are added, the following cells can be changed from "raw" to "code" and executed.

### 3D customized choropleth

not actually intended to be attractive; no analogue in `explore`